In [49]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np

import dataloaders
from importlib import reload
import config
reload(config)
from config import INPUT_SIZE, NUM_CLASSES, HIDDEN_SIZE, N_EPOCHS, LR, LR_DECAY, REG

In [2]:
# Weight initilization
#TO DO: We may try different initialization such as Xavier
def weights_init(m):
    if type(m) == nn.Linear:
        m.weight.data.normal_(0.0, 1e-3)
        m.bias.data.fill_(0.)

def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device: %s'%device)

Using device: cpu


In [4]:
# Load the dataset
train_loader, val_loader, test_loader = dataloaders.create_dataloaders()

Loading train segmentations: 100%|██████████| 8012/8012 [00:06<00:00, 1234.19it/s]


Loading complete, some files (0) were not found: []


Loading test images: 100%|██████████| 2003/2003 [00:09<00:00, 206.11it/s]


Loading complete, some files (0) were not found: []


Loading test images: 100%|██████████| 1512/1512 [00:07<00:00, 211.71it/s]

Loading complete, some files (1) were not found: ['data\\HAM10000_images_test\\ISIC_0035068.jpg']


In [43]:
class CNN(nn.Module):
    def __init__(self, input_size, hidden_layers, num_classes, norm_layer=None):
        super(CNN, self).__init__()
        layers = []

        for l in range(len(hidden_layers)):
            input_size = input_size if l==0 else hidden_layers[l-1]
            layers.append(nn.Conv2d(in_channels=input_size, out_channels=hidden_layers[l], kernel_size=3, stride=1, padding=1))
            #if (norm_layer == 'BN'):
            layers.append(nn.BatchNorm2d(hidden_layers[l]))
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.1))
        layers.append(nn.Flatten()) #Flattening the images for the linear model
        layers.append(nn.Linear(input_size, num_classes))

        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.layers(x)
        return out

In [50]:
# TO DO: Add normalization layer
model = CNN(INPUT_SIZE, HIDDEN_SIZE, NUM_CLASSES).to(device)
model.apply(weights_init)
print(model)

CNN(
  (layers): Sequential(
    (0): Conv2d(3, 750, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): Dropout(p=0.1, inplace=False)
    (4): Conv2d(750, 750, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): ReLU()
    (7): Dropout(p=0.1, inplace=False)
    (8): Flatten(start_dim=1, end_dim=-1)
    (9): Linear(in_features=750, out_features=8, bias=True)
  )
)


In [51]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=REG)

In [52]:
for batch_idx, (images, labels, _) in enumerate(train_loader):
    # Print the size of each image in the batch
    print(f"Batch {batch_idx + 1}, Image size: {images.size()}")

Batch 1, Image size: torch.Size([32, 3, 45, 60])
Batch 2, Image size: torch.Size([32, 3, 45, 60])
Batch 3, Image size: torch.Size([32, 3, 45, 60])
Batch 4, Image size: torch.Size([32, 3, 45, 60])
Batch 5, Image size: torch.Size([32, 3, 45, 60])
Batch 6, Image size: torch.Size([32, 3, 45, 60])
Batch 7, Image size: torch.Size([32, 3, 45, 60])
Batch 8, Image size: torch.Size([32, 3, 45, 60])
Batch 9, Image size: torch.Size([32, 3, 45, 60])
Batch 10, Image size: torch.Size([32, 3, 45, 60])
Batch 11, Image size: torch.Size([32, 3, 45, 60])
Batch 12, Image size: torch.Size([32, 3, 45, 60])
Batch 13, Image size: torch.Size([32, 3, 45, 60])
Batch 14, Image size: torch.Size([32, 3, 45, 60])
Batch 15, Image size: torch.Size([32, 3, 45, 60])
Batch 16, Image size: torch.Size([32, 3, 45, 60])
Batch 17, Image size: torch.Size([32, 3, 45, 60])
Batch 18, Image size: torch.Size([32, 3, 45, 60])
Batch 19, Image size: torch.Size([32, 3, 45, 60])
Batch 20, Image size: torch.Size([32, 3, 45, 60])
Batch 21,

In [53]:
for epoch in range(N_EPOCHS):
    model.train()
    for i, (images, labels, _) in enumerate(train_loader):
        print(i)
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images) #Prediction
        loss = loss_function(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

0


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x123750 and 750x8)